# Supervised Learning Demystified: From First Principles to Production Readiness

Welcome! This series bridges ML theory and real-world system design. Each notebook reveals not just how supervised learning works, but how it behaves in production—where stability, monitoring, and business impact matter as much as accuracy.

### What is Supervised Learning? — A Systems Perspective

Supervised learning is like teaching a system to recognize flowers by showing it many examples, each with a name. The system learns to map features (like petal length and sepal width) to labels (the flower’s species). But in production, the challenge is making sure it keeps learning the right names—even as the garden changes.

Below: A simple diagram of the supervised learning process.

---

**Visual Storytelling:**  
Arrows in the diagram below show how data flows from features, through the model, to predictions. In later sections, you’ll see overlays and interactive controls that reveal how noise or drift can blur these boundaries—just as a gardener might mislabel flowers or new species appear.

---

**Real-World Reflection:**  
In real ML systems, stability and monitoring matter as much as accuracy. Always design for change.

In [1]:
# Visual Storytelling: How Supervised Learning Flows in Real Systems

import plotly.graph_objects as go

fig = go.Figure()

# Draw rectangles for each stage with subtle gradients for visual depth
fig.add_shape(type="rect", x0=0.05, y0=0.4, x1=0.25, y1=0.6,
              line=dict(color="RoyalBlue", width=2), fillcolor="LightSkyBlue")
fig.add_shape(type="rect", x0=0.4, y0=0.4, x1=0.6, y1=0.6,
              line=dict(color="MediumPurple", width=2), fillcolor="Lavender")
fig.add_shape(type="rect", x0=0.75, y0=0.4, x1=0.95, y1=0.6,
              line=dict(color="SeaGreen", width=2), fillcolor="PaleGreen")

# Draw arrows between stages (left to right)
fig.add_annotation(x=0.4, y=0.5, ax=0.25, ay=0.5, xref='x', yref='y', axref='x', ayref='y',
                   showarrow=True, arrowhead=3, arrowsize=2, arrowwidth=2, opacity=0.8)
fig.add_annotation(x=0.75, y=0.5, ax=0.6, ay=0.5, xref='x', yref='y', axref='x', ayref='y',
                   showarrow=True, arrowhead=3, arrowsize=2, arrowwidth=2, opacity=0.8)

# Add labels inside boxes with concise, production-focused language
fig.add_annotation(x=0.15, y=0.5, text="<b>Features<br><span style='font-size:12px'>(Inputs: real data)</span></b>", showarrow=False, font=dict(size=16))
fig.add_annotation(x=0.5, y=0.5, text="<b>Model<br><span style='font-size:12px'>(Learns mapping)</span></b>", showarrow=False, font=dict(size=16))
fig.add_annotation(x=0.85, y=0.5, text="<b>Prediction<br><span style='font-size:12px'>(Outputs: decisions)</span></b>", showarrow=False, font=dict(size=16))

fig.update_layout(
    width=750, height=270,
    margin=dict(l=20, r=20, t=30, b=20),
    xaxis=dict(visible=False, range=[0,1]),
    yaxis=dict(visible=False, range=[0,1]),
    plot_bgcolor='#f8f8fa'
)
fig.show()

In [2]:
# Foundation: Import libraries and load a real dataset (Iris) for all demonstrations

import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.datasets import load_iris

# Load the Iris dataset as a DataFrame for realistic, production-relevant examples
iris = load_iris(as_frame=True)
df = iris.frame

# Quick preview: Real data, real features, real-world class labels
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [3]:
# Visualize class balance before exploring features
# For demonstration: simulate a realistic, slightly imbalanced production scenario
# (In real production, class imbalance is common and can cause instability or bias)
class_counts = df['target'].value_counts().sort_index().copy()
# Artificially adjust counts for demonstration (e.g., drift or sampling bias)
class_counts.iloc[0] -= 8   # Fewer class 0
class_counts.iloc[1] += 5   # More class 1
class_counts.iloc[2] += 3   # Slightly more class 2

fig = px.bar(
    x=iris.target_names,
    y=class_counts,
    labels={'x': 'Class', 'y': 'Count'},
    title="Class Distribution in the Iris Dataset (Simulated Production Imbalance)"
)
fig.update_traces(marker_color=["#4F81BD", "#C0504D", "#9BBB59"])
fig.show()

*Note: The Iris dataset is perfectly balanced—each class has exactly 50 samples. In production, class imbalance is common and can cause instability or bias. Always check class balance before modeling.*

In [4]:
# Progressive Disclosure & Interactive Visuals: Feature selectors and noise toggle for 2D/3D plots

import plotly.express as px
import pandas as pd
import numpy as np
from ipywidgets import Dropdown, ToggleButton, HBox, VBox, Output, Button, Layout
from IPython.display import display, Markdown
import IPython

# Feature options for selectors (use full words for clarity)
feature_options = [
    ("Sepal Length", "sepal length (cm)"),
    ("Sepal Width", "sepal width (cm)"),
    ("Petal Length", "petal length (cm)"),
    ("Petal Width", "petal width (cm)")
]

# Responsive dropdowns and button widths for clarity and compactness
dropdown_layout = Layout(width="180px", margin="0 4px 0 0")
button_layout = Layout(width="170px", margin="0 4px 0 0")
toggle_layout = Layout(width="120px", margin="0 4px 0 0")

x_feat = Dropdown(options=feature_options, value="sepal length (cm)", description="X:", layout=dropdown_layout)
y_feat = Dropdown(options=feature_options, value="petal length (cm)", description="Y:", layout=dropdown_layout)
z_feat = Dropdown(options=feature_options, value="petal width (cm)", description="Z:", layout=dropdown_layout)
noise_toggle = ToggleButton(
    value=False,
    description="Add 10% Label Noise",
    button_style='warning',
    tooltip="Toggle to add random label noise (simulates real-world label errors)",
    layout=button_layout
)
plot3d_toggle = ToggleButton(
    value=True,
    description="2D Plot",
    button_style='info',
    tooltip="Switch between 2D and 3D feature visualization",
    layout=toggle_layout
)
reset_btn = Button(description="Reset View", button_style='info', layout=toggle_layout)

out = Output()

def get_noisy_df(df, noise_on):
    if not noise_on:
        return df.copy()
    noisy_df = df.copy()
    np.random.seed(42)
    noise_idx = np.random.choice(noisy_df.index, size=int(0.1 * len(noisy_df)), replace=False)
    noisy_df.loc[noise_idx, 'target'] = np.random.choice(noisy_df['target'].unique(), size=len(noise_idx))
    return noisy_df

def safe_hover(val):
    # Avoid 'null' in hover: show '—' for missing values
    return "—" if pd.isnull(val) else val

def plot_features(x, y, z, noise_on, plot3d):
    noisy_df = get_noisy_df(df, noise_on)
    noisy_df = noisy_df.copy()
    for col in [x, y, z]:
        if col in noisy_df:
            noisy_df[col] = noisy_df[col].apply(safe_hover)
    # Use all available width in the notebook cell
    # Get notebook width in pixels (fallback to 1000 if not available)
    try:
        from IPython.display import Javascript, display as jsdisplay
        jsdisplay(Javascript("""
        if (!window._notebook_width) {
            window._notebook_width = document.querySelector('.jp-NotebookPanel-notebook, .notebook-container, .jp-Notebook').offsetWidth;
        }
        """))
        notebook_width = IPython.get_ipython().user_ns.get('_notebook_width', 1000)
    except Exception:
        notebook_width = 1000
    # Use 98% of available width, but cap at 1400px for very wide screens
    plot_width = min(1400, int(0.98 * notebook_width))
    plot_height = 500

    if plot3d:
        fig = px.scatter_3d(
            noisy_df,
            x=x, y=y, z=z,
            color=noisy_df["target"].astype(str),
            labels={
                "color": "Class",
                x: next(label for label, value in feature_options if value == x),
                y: next(label for label, value in feature_options if value == y),
                z: next(label for label, value in feature_options if value == z)
            },
            title="Iris Dataset: 3D Feature Visualization" + (" (Noisy Labels)" if noise_on else "")
        )
        fig.update_traces(
            marker=dict(size=8, opacity=0.85, line=dict(width=0.5, color="#888")),
            hovertemplate=(
                f"{next(label for label, value in feature_options if value == x)}: %{{x}}<br>"
                f"{next(label for label, value in feature_options if value == y)}: %{{y}}<br>"
                f"{next(label for label, value in feature_options if value == z)}: %{{z}}<br>"
                "Class: %{marker.color}"
            )
        )
        initial_camera = dict(eye=dict(x=1.7, y=1.7, z=1.1), center=dict(x=0, y=0, z=0))
        fig.update_layout(
            width=plot_width, height=plot_height,
            scene_camera=initial_camera,
            scene_dragmode='turntable',
            margin=dict(l=10, r=10, t=30, b=10),
            scene=dict(
                xaxis_title=next(label for label, value in feature_options if value == x),
                yaxis_title=next(label for label, value in feature_options if value == y),
                zaxis_title=next(label for label, value in feature_options if value == z),
                bgcolor="#f4f5fa"
            ),
            paper_bgcolor="#f4f5fa",
            plot_bgcolor="#f4f5fa"
        )
    else:
        fig = px.scatter(
            noisy_df,
            x=x, y=y,
            color=noisy_df["target"].astype(str),
            labels={
                "color": "Class",
                x: next(label for label, value in feature_options if value == x),
                y: next(label for label, value in feature_options if value == y)
            },
            title="Iris Dataset: 2D Feature Visualization" + (" (Noisy Labels)" if noise_on else "")
        )
        fig.update_traces(
            marker=dict(size=10, opacity=0.85, line=dict(width=0.5, color="#888")),
            hovertemplate=(
                f"{next(label for label, value in feature_options if value == x)}: %{{x}}<br>"
                f"{next(label for label, value in feature_options if value == y)}: %{{y}}<br>"
                "Class: %{marker.color}"
            )
        )
        fig.update_layout(
            width=plot_width, height=plot_height,
            margin=dict(l=10, r=10, t=30, b=10),
            xaxis_title=next(label for label, value in feature_options if value == x),
            yaxis_title=next(label for label, value in feature_options if value == y),
            plot_bgcolor="#f4f5fa",
            paper_bgcolor="#f4f5fa"
        )
    return fig

def update_plot(*args):
    plot3d_toggle.description = "2D Plot" if plot3d_toggle.value else "3D Plot"
    out.clear_output(wait=True)
    with out:
        fig = plot_features(
            x_feat.value, y_feat.value, z_feat.value, noise_toggle.value, plot3d_toggle.value
        )
        fig.show()
        if noise_toggle.value:
            display(Markdown(
                "<span style='color:#b36b00'><b>Architect’s Note:</b> Label noise blurs class boundaries. "
                "In production, this means more misclassifications and less trust in predictions. "
                "Always monitor label quality and set up alerts for drift or anomalies.</span>"
            ))
        else:
            display(Markdown(
                "<span style='color:#555; font-size:12px'><b>Tip:</b> Try toggling noise or switching features to see how class separability changes. "
                "In production, feature selection and data quality are key to robust ML systems.</span>"
            ))

def reset_view(_):
    update_plot()

# Attach callbacks
x_feat.observe(update_plot, names='value')
y_feat.observe(update_plot, names='value')
z_feat.observe(update_plot, names='value')
noise_toggle.observe(update_plot, names='value')
plot3d_toggle.observe(update_plot, names='value')
reset_btn.on_click(reset_view)

# Controls: compact, no vertical scrollbar, responsive
controls = HBox(
    [x_feat, y_feat, z_feat, plot3d_toggle, noise_toggle, reset_btn],
    layout=Layout(justify_content="flex-start", align_items="center", gap="4px", margin="0 0 8px 0", width="100%")
)
display(VBox([controls, out], layout=Layout(width="100%")))
update_plot()

*Figure: Each point is a sample. Color shows class. Patterns hint at which features separate classes best.*

In production, supervised learning is about far more than just accuracy. Data drift, label quality, and seamless system integration all shape model stability and business impact. Robust ML systems continuously monitor these factors, trigger retraining when needed, and adapt as data and requirements evolve. This is where engineering rigor meets architectural depth—ensuring models remain reliable, scalable, and aligned with real-world change.

In [5]:
# Visualize feature interactions: 2D scatter plots for two feature pairs
import plotly.subplots as sp
import plotly.express as px

fig = sp.make_subplots(rows=1, cols=2, subplot_titles=("Sepal Len vs Petal Len", "Petal Len vs Petal Wid"))

scatter1 = px.scatter(
    df,
    x="sepal length (cm)",
    y="petal length (cm)",
    color=df["target"].astype(str),
    labels={"color": "Class"},
)
scatter2 = px.scatter(
    df,
    x="petal length (cm)",
    y="petal width (cm)",
    color=df["target"].astype(str),
    labels={"color": "Class"},
)

for trace in scatter1.data:
    fig.add_trace(trace, row=1, col=1)
for trace in scatter2.data:
    fig.add_trace(trace, row=1, col=2)

fig.update_layout(
    width=1000, height=400,
    title_text="Iris Dataset: Feature Interactions (2D Views)",
    showlegend=False,
    margin=dict(l=20, r=20, t=40, b=20)
)
fig.show()

*Figure: Adding label noise makes class boundaries less clear. This simulates real-world data issues.*

## Real-World Reflection: Data Quality in Production

In real systems, label noise can come from human error, ambiguous cases, or process drift. This degrades model performance and can cause instability. Production ML systems must include data validation, monitoring, and retraining triggers to handle these issues.

---
**What’s Next?**

In the next notebook, we’ll explore how cost functions and gradients drive model learning—and why their behavior is critical for system stability and scaling in production.